# Проверка качества данных: Источник H2O (Факторы проблемности)

**Таблица:** `sandbox_ai.tmp_h20_fp_svy`  
**Содержание:** данные о факторах проблемности клиентов банка

## Загрузка данных

In [ ]:
import pandas as pd
import numpy as np

FILE_PATH = "../sources/data_h20.xlsx"

FIELD_DESCRIPTIONS = {
    "inn": "ИНН клиента. 10 цифр для юридических лиц, 12 цифр для ИП/КФХ",
    "client_name": "Наименование компании клиента (включая организационно-правовую форму)",
    "cdi_id": "Идентификатор клиента во внутренних источниках банка",
    "legal_form": "Организационно-правовая форма компании клиента",
    "rf": "Региональный филиал банка, за которым закреплён клиент",
    "segment": "Сегмент бизнеса клиента по обороту",
    "mon_type": "Вид мониторинга клиента (допустимо NULL)",
    "h20_fp_id": "Идентификатор карточки фактора проблемности в системе H2O",
    "crm_fp_id": "Идентификатор карточки фактора проблемности в системе CRM",
    "ref_book_fp_id": "Идентификатор фактора проблемности в справочнике факторов",
    "fp_start_date": "Дата выявления фактора проблемности (ДД.ММ.ГГГГ)",
}

ID_COLUMNS = ["h20_fp_id", "crm_fp_id"]

In [ ]:
df = pd.read_excel(FILE_PATH, dtype=str)
df.columns = df.columns.str.strip()

print(f"Строк: {len(df):,}")
print(f"Колонок: {len(df.columns)}")
df.head()

## 1. Список всех полей с описанием

In [ ]:
fields_data = []
for i, col in enumerate(df.columns, start=1):
    desc = FIELD_DESCRIPTIONS.get(col, "(описание отсутствует)")
    fields_data.append({"\u2116": i, "Поле": col, "Описание": desc})

df_fields = pd.DataFrame(fields_data)
df_fields.style.hide(axis="index")

## 2. Пропуски и пустые значения

In [ ]:
total = len(df)

nulls_data = []
for col in df.columns:
    s = df[col]
    cnt_nan = s.isna().sum()
    s_str = s.dropna().astype(str).str.strip().str.lower()
    cnt_none = (s_str == "none").sum()
    cnt_nat = (s_str == "nat").sum()
    cnt_empty = (s_str == "").sum()
    cnt_total = cnt_nan + cnt_none + cnt_nat + cnt_empty
    pct = cnt_total / total * 100

    nulls_data.append({
        "Поле": col,
        "NaN": cnt_nan,
        "None": cnt_none,
        "NaT": cnt_nat,
        "Пустые": cnt_empty,
        "Итого": cnt_total,
        "%": round(pct, 1),
    })

df_nulls = pd.DataFrame(nulls_data)

df_nulls.style.hide(axis="index")

## 3. Полные дубликаты

In [ ]:
dup_mask = df.duplicated(keep=False)
n_dup_rows = dup_mask.sum()
n_extra_copies = df.duplicated(keep="first").sum()

print(f"Всего строк в таблице: {len(df):,}")
print(f"Строк, участвующих в дублировании: {n_dup_rows}")
print(f"Из них лишних копий: {n_extra_copies}")

if n_dup_rows > 0:
    dup_df = df[dup_mask].copy()
    dup_df["_group"] = dup_df.groupby(list(df.columns)).ngroup()
    dup_df["_count"] = dup_df.groupby("_group")["_group"].transform("count")
    dup_display = (
        dup_df.drop_duplicates(subset="_group")
        .sort_values("_group")
        .rename(columns={"_count": "Повторений"})
        .drop(columns="_group")
        .reset_index(drop=True)
    )
    print(f"\nДублирующиеся строки:")
    display(dup_display)
else:
    print("\nПолных дубликатов не обнаружено.")

## 4. Уникальность идентификаторов

In [ ]:
for id_col in ID_COLUMNS:
    if id_col not in df.columns:
        continue

    clean = df[id_col].dropna().astype(str).str.strip()
    clean = clean[clean != ""]

    n_total = len(clean)
    n_unique = clean.nunique()
    n_dupes = clean.duplicated().sum()

    print(f"--- {id_col} ---")
    print(f"  Заполнено: {n_total:,}")
    print(f"  Уникальных: {n_unique:,}")
    print(f"  Повторяющихся: {n_dupes}")

    if n_dupes > 0:
        dup_ids = clean[clean.duplicated(keep=False)]
        dup_detail = dup_ids.value_counts().reset_index()
        dup_detail.columns = [id_col, "Кол-во"]
        print(f"\n  Повторяющиеся значения:")
        display(dup_detail)
    print()

## 5. Проверка ИНН

In [ ]:
inn_col = df["inn"].dropna().astype(str).str.strip()
inn_col = inn_col[(inn_col != "") & (inn_col.str.lower() != "nan")]

inn_counts = inn_col.value_counts()
n_total = len(inn_col)
n_unique = inn_col.nunique()
n_repeat = (inn_counts > 1).sum()

print(f"Всего записей с ИНН: {n_total:,}")
print(f"Уникальных ИНН: {n_unique:,}")
print(f"ИНН, встречающихся более 1 раза: {n_repeat:,}")

if n_repeat > 0:
    repeat_df = (
        inn_counts[inn_counts > 1]
        .reset_index()
    )
    repeat_df.columns = ["ИНН", "Кол-во записей"]
    print(f"\nПовторяющиеся ИНН:")
    display(repeat_df)